# Exploratory Analisys

## Context

As discussed in previous notebooks, we will use a tree-based model to predict the residuals of our harmonic regression. The most common options are Random Forest — which we also used for feature selection — and XGBoost. Both models are appropriate and have distinct advantages and limitations in this context. To determine which model to use, we will compare their predictive performance. Because their internal structures differ, they may perform differently on the same feature set, therefore, we first define the feature set for each model and then evaluate and compare their forecasting accuracy.

**Data Source**
The data used in this notebook was extracted from the notebook *eda-non-linear.ipynb*

- **Data:** 19/08/2025
- **Localização:** ../data/wrangle

## Set up

### Libraries

In [41]:
## Base
import os
import pickle
import numpy as np
import pandas as pd

## Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Model
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.inspection import permutation_importance
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold
import shap

In [5]:
# Funções criadas
import sys
from pathlib import Path
sys.path.insert(1, Path.cwd().parents[0].as_posix())

from src.ts_utils import *

from config import *

In [6]:
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#003366'])

## Data

The first step for comparison is to define our training data. We will use **weather\_linear\_resids** and **weather\_validation**, ensuring that after creating the features we still have sufficient data to train and test our models.

In [9]:
train_df = pd.read_parquet(os.path.join(DATA_PATH_WRANGLE, 'weather_linear_resids.parquet')).set_index('time')
valid_df = pd.read_parquet(os.path.join(DATA_PATH_WRANGLE, 'weather_validation.parquet')).set_index('time')

df = pd.concat([train_df, valid_df])
display(df.head())
df.info()

,tavg,prcp,snow,wspd,pres,tamp,wcardinal,y_hat,resid
time,,,,,,,,,
2015-08-19,295.8,0.0,0.0,6.6,1004.8,8.3,Southwest,297.100872,-1.300872
2015-08-20,292.8,0.0,0.0,6.6,1010.2,8.3,West,297.001835,-4.201835
2015-08-21,295.8,0.0,0.0,3.9,1017.5,13.2,South,296.899932,-1.099932
2015-08-22,297.1,0.0,0.0,4.1,1017.2,9.4,South,296.795190,0.304810
2015-08-23,296.1,0.5,0.0,5.2,1011.3,7.8,Southwest,296.687632,-0.587632


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2548 entries, 2015-08-19 to 2022-08-09
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   tavg       2548 non-null   float64 
 1   prcp       2548 non-null   float64 
 2   snow       2548 non-null   float64 
 3   wspd       2548 non-null   float64 
 4   pres       2548 non-null   float64 
 5   tamp       2548 non-null   float64 
 6   wcardinal  2548 non-null   category
 7   y_hat      1818 non-null   float64 
 8   resid      1818 non-null   float64 
dtypes: category(1), float64(8)
memory usage: 182.0 KB


In [10]:
feature_imp = pd.read_parquet(DATA_PATH_WRANGLE / "feature_importance.parquet")
display(feature_imp.head())
feature_imp.info()

,feature,importance
0,snow_387,0.025868
1,tavg_668,0.013156
2,tavg_667,0.009291
3,snow_388,0.007665
4,tavg_669,0.006650


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   feature     750 non-null    object 
 1   importance  750 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.8+ KB


## Target Creation

Only our **weather\_linear\_resids** data has the **resid** column filled. Since this is our target feature, we need to complete it before proceeding. To do so, we must train the harmonic regression. To simplify the process, we will use the model from the **eda-linear.ipynb** notebook.

In [11]:
n = len(df)
t = np.arange(n)

omega1 = (2 * np.pi / 365.25)
omega2 = 2 * (2 * np.pi / 365.25)
omega3 = 3 * (2 * np.pi / 365.25)

y = df['tavg'].copy()
X = np.column_stack([
    np.ones(n),                 
    np.sin(omega1*t),
    np.cos(omega1*t),
    np.cos(omega2*t),
    np.cos(omega3*t)
])

model = QuantReg(y, X)
res = model.fit(q=0.5)

df['y_hat'] = res.predict(X)
df['resid'] = res.resid

In [12]:
df.tail()

,tavg,prcp,snow,wspd,pres,tamp,wcardinal,y_hat,resid
time,,,,,,,,,
2022-08-05,300.3,0.0,0.0,2.7,1016.9,8.3,East,298.473297,1.826703
2022-08-06,303.2,0.0,0.0,4.1,1016.3,10.0,Southwest,298.423329,4.776671
2022-08-07,300.5,14.7,0.0,3.4,1015.8,3.2,Southwest,298.369543,2.130457
2022-08-08,297.8,3.0,0.0,4.9,1013.1,8.2,West,298.311971,-0.511971
2022-08-09,295.3,0.0,0.0,3.8,1018.3,7.4,Northeast,298.250643,-2.950643


## Feature Creation

After creating the target for our model, we need to generate the features. All the 750 features previously selected.

### Diff Features

In [13]:
name_features_diff = feature_imp.loc[feature_imp['feature'].str.contains('diff'), 'feature']

list_feature_diff = []
for name in tqdm(name_features_diff, desc='Diff'):
    # Getting information from name
    col, shift, _ = name.split('_')

    # Feature Creation
    df_feature = df[[col]].shift(int(shift)).diff()
    df_feature.columns = [name]

    # Saving Feature
    list_feature_diff.append(df_feature)

Diff: 100%|█████████████████████████████████████████████████████████████████████████████████████| 113/113 [00:00<00:00, 500.86it/s]


### Shift Features

In [14]:
name_features = feature_imp.loc[(~feature_imp['feature'].str.contains('diff')) & 
                                (~feature_imp['feature'].str.contains('wcardinal')) & 
                                (~feature_imp['feature'].str.contains('month')) & 
                                (~feature_imp['feature'].str.contains('day')) & 
                                (~feature_imp['feature'].str.contains('season')) & 
                                (feature_imp['feature'].str.contains('_')), 'feature']

list_feature = []
for name in tqdm(name_features, desc='Diff'):
    # Getting information from name
    col, shift = name.split('_')

    # Feature Creation
    df_feature = df[[col]].shift(int(shift))
    df_feature.columns = [name]

    # Saving Feature
    list_feature.append(df_feature)

Diff: 100%|████████████████████████████████████████████████████████████████████████████████████| 632/632 [00:00<00:00, 1483.46it/s]


### Category Features

#### Generating wcardinal

In [15]:
c_feature_list = []
for step in range(365, 731):
    if any(feature_imp['feature'].str.contains(f'wcardinal_{step}')):
        cardinal_feature = df[['wcardinal']].shift(step)
        cardinal_feature.columns = [f'wcardinal_{step}']
        c_feature_list.append(cardinal_feature)
c_feature_df = pd.concat(c_feature_list, axis=1)

#### Generating date

In [16]:
df['day'] = df.index.day
df['month'] = df.index.strftime('%b')

df['season'] = 'Spring'
df.loc[(df.index.month >= 6) & (df.index.month <= 8), 'season'] = 'Summer'
df.loc[(df.index.month >= 9) & (df.index.month <= 11), 'season'] = 'Autumn'
df.loc[(df.index.month == 12) & (df.index.month <= 2), 'season'] = 'December'

In [17]:
df = pd.concat([df, c_feature_df], axis=1)
df_date = pd.get_dummies(df.dropna(), columns=['day', 'month', 'season'] + c_feature_df.columns.tolist())
df_date = pd.concat([df, df_date], axis=1)
df_date.head()

,tavg,prcp,snow,wspd,pres,tamp,wcardinal,y_hat,resid,day,...,wcardinal_564_Southwest,wcardinal_564_West,wcardinal_698_East,wcardinal_698_North,wcardinal_698_Northeast,wcardinal_698_Northwest,wcardinal_698_South,wcardinal_698_Southeast,wcardinal_698_Southwest,wcardinal_698_West
time,,,,,,,,,,,,,,,,,,,,,
2015-08-19,295.8,0.0,0.0,6.6,1004.8,8.3,Southwest,297.462105,-1.662105,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-20,292.8,0.0,0.0,6.6,1010.2,8.3,West,297.362314,-4.562314,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-21,295.8,0.0,0.0,3.9,1017.5,13.2,South,297.259120,-1.459120,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-22,297.1,0.0,0.0,4.1,1017.2,9.4,South,297.152550,-0.052550,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-08-23,296.1,0.5,0.0,5.2,1011.3,7.8,Southwest,297.042633,-0.942633,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
feature_list_category = []
for col in df_date.columns:
    if col in feature_imp['feature'].tolist():
        feature_list_category.append(df_date[col])

## DataFrame Consolidation

In [19]:
df_model = pd.concat(list_feature_diff + list_feature +feature_list_category, axis=1).dropna()
df_model.shape

(1817, 750)

# Modeling Selection

The most important task in this notebook is to define the comparison methodology. The first step is to establish feature selection for each model. To do this, we will define the training, validation, and test sets, and then train each model using multiple features. This approach allows us to generate a performance curve and identify the feature range most worthy of further investigation. The evaluation metric we will use is RMSE. We will use the RMSE of our target (which is already a residual) as the threshold

## Data Split

In [20]:
X = df_model.astype(int).copy()
y = df['resid'].reindex(df_model.index)

threshold_rsme = np.sqrt(np.mean(y**2))
threshold_rsme

np.float64(4.796181263438479)

## Feature Selection
To begin our selection, we create incremental sets of features. This allows us to train and compare the metric for each set, helping us evaluate how well our model is able to address the problem

In [16]:
feature_imp.sort_values('importance', ascending=False, inplace=True)
feature_collection = []

for n_features in range(10, 751, 20):
    feature_collection.append(feature_imp.head(n_features)['feature'].tolist())

### Random Forrest
Since Random Forest is usually fast to train, especially with that amounts of data, we will use cross-validation to obtain a more reliable measure of how well the model adapts

In [16]:
feature_score = {}

for feature_set in tqdm(feature_collection):
    rf = RandomForestRegressor(random_state=25, n_jobs=-2)
    cv_results = cross_validate(rf, X[feature_set], y, cv=10,
                               scoring='neg_root_mean_squared_error')

    feature_score[len(feature_set)] = [-np.mean(cv_results['test_score']), -np.quantile(cv_results['test_score'], 0.25), -np.quantile(cv_results['test_score'], 0.75)]

 63%|████████████████████████████████████████████████████████████████████▊                                        | 24/38 [09:02<05:16, 22.59s/it]

KeyboardInterrupt


KeyboardInterrupt



In [ ]:
geral_feature_score = pd.DataFrame(feature_score, index=['RMSE', '1Q', '3Q']).T

plt.figure(figsize=(15, 4))
sns.scatterplot(x=geral_feature_score.index, y=geral_feature_score['RMSE'])
sns.lineplot(x=geral_feature_score.index, y=geral_feature_score['RMSE'])
plt.fill_between(x=geral_feature_score.index, y1=geral_feature_score['1Q'], 
                 y2=geral_feature_score['3Q'], color='gray', alpha=0.3)

plt.axhline(y=threshold_rsme, color='red', linestyle='--')

plt.xlabel('Feature Numbers')
plt.grid()
plt.show()

The blue line represents the average RMSE, while the grey area represents the range between the first and third quartiles.

Above, we can see how the Random Forest model performed for each feature set relative to the threshold. The model did not fit well, indicating that even with hyperparameter tuning, it is unlikely to achieve performance that justifies its use.

### XGBoost

The XGBoost model structure differs from Random Forest, so it may produce different results. To allow comparison, we applied the same method used for Random Forest.

In [ ]:
xgboost_feature_score = {}

for feature_set in tqdm(feature_collection):    
    kf = KFold(n_splits=10, shuffle=True, random_state=25)
    rmse_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_idx][feature_set], X.iloc[val_idx][feature_set]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
        model = XGBRegressor(
            objective="reg:squarederror",
            random_state=25
        )
        model.fit(X_train, y_train)
    
        # Predict & evaluate
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        rmse_scores.append(rmse)

    xgboost_feature_score[len(feature_set)] = [np.mean(rmse_scores), np.quantile(rmse_scores, 0.25), np.quantile(rmse_scores, 0.75)]

In [ ]:
geral_xgboost_feature_score = pd.DataFrame(xgboost_feature_score, index=['RMSE', '1Q', '3Q']).T

plt.figure(figsize=(15, 4))
sns.scatterplot(x=geral_xgboost_feature_score.index, y=geral_xgboost_feature_score['RMSE'])
sns.lineplot(x=geral_xgboost_feature_score.index, y=geral_xgboost_feature_score['RMSE'])
plt.fill_between(x=geral_xgboost_feature_score.index, y1=geral_xgboost_feature_score['1Q'], 
                 y2=geral_xgboost_feature_score['3Q'], color='gray', alpha=0.3)

plt.axhline(y=threshold_rsme, color='red', linestyle='--')

plt.xlabel('Feature Numbers')
plt.grid()
plt.show()

Here, we can clearly see that XGBoost outperformed Random Forest. Based on this, XGBoost was chosen as the model. The features were initially selected based on Random Forest performance; however, given the large performance gap between XGBoost and Random Forest, it is possible that the selected features are not optimal for XGBoost. Therefore, we will re-define the feature relevance order.


## XGBoost SHAP
Let’s begin the new feature selection using the SHAP method. Below, we can see all the possible features.

In [21]:
feature_eng = pd.read_parquet(DATA_PATH_WRANGLE / 'possible_features.parquet')
feature_eng.head()

,tavg_365,tavg_366,tavg_367,tavg_368,tavg_369,tavg_370,tavg_371,tavg_372,tavg_373,tavg_374,...,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep,season_Autumn,season_Spring,season_Summer,resid
time,,,,,,,,,,,,,,,,,,,,,
2017-08-19,298.0,299.0,299.3,298.6,296.8,298.8,299.5,299.5,302.8,301.5,...,False,False,False,False,False,False,False,False,True,-0.251713
2017-08-20,296.8,298.0,299.0,299.3,298.6,296.8,298.8,299.5,299.5,302.8,...,False,False,False,False,False,False,False,False,True,1.848760
2017-08-21,293.3,296.8,298.0,299.0,299.3,298.6,296.8,298.8,299.5,299.5,...,False,False,False,False,False,False,False,False,True,2.252086
2017-08-22,294.4,293.3,296.8,298.0,299.0,299.3,298.6,296.8,298.8,299.5,...,False,False,False,False,False,False,False,False,True,-0.241761
2017-08-23,296.6,294.4,293.3,296.8,298.0,299.0,299.3,298.6,296.8,298.8,...,False,False,False,False,False,False,False,False,True,-2.832804


For the SHAP method, we first need to train the XGBoost model using all available features.

In [31]:
X_train, X_val, y_train, y_val = train_test_split(feature_eng.drop(columns=['resid']), feature_eng[['resid']], 
                                                  test_size=0.33, random_state=25)

# Train model
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


With the model trained, we can measure the relevance of each feature. Based on this ranking, we select the top 100 features. In the next step, we will test combinations of these 100 features to determine which set achieves the best performance.

In [32]:
# Create the explainer
explainer = shap.TreeExplainer(xgb)

# Calculate SHAP values for the test set
shap_values = explainer.shap_values(X_val)

# Calculate Metric
mean_shap = np.abs(shap_values).mean(axis=0)
feature_importance = dict(zip(X_val.columns, mean_shap))
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

# Top 500 features
top_features = [f for f, v in sorted_features[:500]]

With this list, we can proceed to the next step: testing each feature combination to identify the optimal set, after which we will move on to hyperparameter tuning.

In [43]:
with open("../data/wrangle/top_100_features.pkl", "wb") as f:
    pickle.dump(top_features, f)